In [37]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np

# List all CSV files in the directory
data_dir = './hungergames/'
data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]

# Print out the files we'll be working with
print("Found the following CSV files:")
for file in data_files:
    print(f"  - {os.path.basename(file)}")


Found the following CSV files:
  - additional-quiz-results.csv
  - student-records.csv
  - course-assignments.csv
  - login-activity.csv
  - assessment-scores.csv


In [38]:
for file in data_files:
  print(file)

./hungergames/additional-quiz-results.csv
./hungergames/student-records.csv
./hungergames/course-assignments.csv
./hungergames/login-activity.csv
./hungergames/assessment-scores.csv


## Discovery

In [39]:
# Examine the data sources
for file in data_files:
    df = pd.read_csv(file)
    print(f"File: {file}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Missing values:\n{df.isnull().sum()}")
    print("---")


File: ./hungergames/additional-quiz-results.csv
Shape: (32, 5)
Columns: ['Name', 'Quiz ID', 'Time Taken', 'Mark', 'Status']
Missing values:
Name          0
Quiz ID       0
Time Taken    1
Mark          0
Status        0
dtype: int64
---
File: ./hungergames/student-records.csv
Shape: (16, 6)
Columns: ['student_id', 'first_name', 'last_name', 'district', 'registration_date', 'status']
Missing values:
student_id           1
first_name           0
last_name            6
district             0
registration_date    0
status               1
dtype: int64
---
File: ./hungergames/course-assignments.csv
Shape: (9, 6)
Columns: ['assignment_id', 'title', 'due_date', 'max_points', 'weight', 'category']
Missing values:
assignment_id    0
title            0
due_date         0
max_points       0
weight           0
category         0
dtype: int64
---
File: ./hungergames/login-activity.csv
Shape: (16, 6)
Columns: ['id', 'first_login', 'last_login', 'logins_count', 'avg_session_minutes', 'device_type']
Mi

Structuring

In [40]:
# Standardize column names
def standardize_columns(df):
    return df.rename(columns={
        'student_id': 'student_id',
        'stu_id': 'student_id',
        'id': 'student_id',
        'grade': 'score',
        'mark': 'score',
        'points': 'score'
    })

# Apply to all dataframes
for df in dfs:
  print("Before:", df.columns.tolist())

dfs = [standardize_columns(pd.read_csv(file)) for file in data_files]
for df in dfs:
    print("After:", df.columns.tolist())


Before: ['Name', 'Quiz ID', 'Time Taken', 'Mark', 'Status']
Before: ['student_id', 'first_name', 'last_name', 'district', 'registration_date', 'status']
Before: ['assignment_id', 'title', 'due_date', 'max_points', 'weight', 'category']
Before: ['student_id', 'first_login', 'last_login', 'logins_count', 'avg_session_minutes', 'device_type']
Before: ['student_id', 'assignment_id', 'score', 'submission_time', 'feedback']
After: ['Name', 'Quiz ID', 'Time Taken', 'Mark', 'Status']
After: ['student_id', 'first_name', 'last_name', 'district', 'registration_date', 'status']
After: ['assignment_id', 'title', 'due_date', 'max_points', 'weight', 'category']
After: ['student_id', 'first_login', 'last_login', 'logins_count', 'avg_session_minutes', 'device_type']
After: ['student_id', 'assignment_id', 'score', 'submission_time', 'feedback']


Cleaning

In [45]:
for df in dfs:
    print("After:", df.columns.tolist())
for df in dfs:
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(df.head(3))
    print("---")

# Handle student ID inconsistencies
def clean_student_id(df):
    # First, identify which column contains the student ID
    id_columns = ['student_id', 'stu_id', 'id']
    id_col = None

    for col in id_columns:
        if col in df.columns:
            id_col = col
            break

    # If no ID column is found, check if there's a 'Name' column (for quiz_results.csv)
    if id_col is None:
        if 'Name' in df.columns:
            # We'll handle this separately for now
            return df
        else:
            print(f"Warning: No ID column found in dataframe with columns: {df.columns}")
            return df

    # Make a copy of the dataframe to avoid the SettingWithCopyWarning
    df = df.copy()

    # Ensure ID is string type
    df[id_col] = df[id_col].astype(str)

    # Remove prefixes and standardize format
    df[id_col] = df[id_col].str.replace('S-', '')
    df[id_col] = df[id_col].str.replace('ID', '')
    df[id_col] = df[id_col].str.strip()

    # Rename the column to the standard 'student_id'
    if id_col != 'student_id':
        df = df.rename(columns={id_col: 'student_id'})

    return df

# Clean timestamps
def clean_timestamps(df):
    # Make a copy of the dataframe
    df = df.copy()

    # Convert timestamp columns to datetime
    time_cols = [col for col in df.columns
                if any(term in col.lower() for term in ['time', 'date', 'login'])]

    for col in time_cols:
        if col in df.columns:  # Verify column exists
            df[col] = pd.to_datetime(df[col], errors='coerce')

    return df

# Apply cleaning functions one at a time with error handling
clean_dfs = []
for i, df in enumerate(dfs):
    try:
        # Print which file we're processing
        print(f"Cleaning file: {os.path.basename(data_files[i])}")

        # Apply cleaning functions
        cleaned = clean_student_id(df)
        cleaned = clean_timestamps(cleaned)

        clean_dfs.append(cleaned)
    except Exception as e:
        print(f"Error cleaning {os.path.basename(data_files[i])}: {e}")
        # Add the original df to maintain indexing
        clean_dfs.append(df)
    print("======")

for df in clean_dfs:
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(df.head(3))
    print("---")

After: ['Name', 'Quiz ID', 'Time Taken', 'Mark', 'Status']
After: ['student_id', 'first_name', 'last_name', 'district', 'registration_date', 'status']
After: ['assignment_id', 'title', 'due_date', 'max_points', 'weight', 'category']
After: ['student_id', 'first_login', 'last_login', 'logins_count', 'avg_session_minutes', 'device_type']
After: ['student_id', 'assignment_id', 'score', 'submission_time', 'feedback']
Shape: (32, 5)
Columns: ['Name', 'Quiz ID', 'Time Taken', 'Mark', 'Status']
               Name Quiz ID           Time Taken  Mark    Status
0  Katniss Everdeen    Q101  2022-09-20 10:15:00    45  Complete
1     Peeta Mellark    Q101  2022-09-20 10:30:00    42  Complete
2    Gale Hawthorne    Q101  2022-09-20 10:45:00    38  Complete
---
Shape: (16, 6)
Columns: ['student_id', 'first_name', 'last_name', 'district', 'registration_date', 'status']
  student_id first_name  last_name district registration_date  status
0      S-001    Katniss   Everdeen       12        2022-09-01  A

Enriching

In [49]:
# Special handling for quiz_results.csv which uses names instead of IDs
quiz_df = None
for i, df in enumerate(clean_dfs):
    if 'Name' in df.columns and 'Mark' in df.columns:
        quiz_df = df.copy()

        # Split the Name column into first_name and last_name
        quiz_df[['first_name', 'last_name']] = quiz_df['Name'].str.split(' ', n=1, expand=True)

        # Rename Mark to score for consistency
        quiz_df = quiz_df.rename(columns={'Mark': 'score', 'Quiz ID': 'assignment_id'})

        # Remove this from clean_dfs as we'll handle it separately
        clean_dfs[i] = None

# Remove None entries from clean_dfs
clean_dfs = [df for df in clean_dfs if df is not None]

# Get the student records dataframe which has first_name and last_name
student_records_df = None
for df in clean_dfs:
    if 'first_name' in df.columns and 'last_name' in df.columns:
        student_records_df = df
        break

# If we have both quiz_df and student_records_df, we can match the quiz results to student IDs
if quiz_df is not None and student_records_df is not None:
    # Merge on first_name and last_name
    quiz_df = pd.merge(
        quiz_df,
        student_records_df[['student_id', 'first_name', 'last_name']],
        on=['first_name', 'last_name'],
        how='left'
    )

    # Add this back to clean_dfs
    clean_dfs.append(quiz_df)

# Start with the student records as our base
base_df = student_records_df if student_records_df is not None else clean_dfs[0]
merged_df = base_df.copy()

# Merge the other dataframes one by one
for df in clean_dfs:
    if df is not base_df and 'student_id' in df.columns:  # Skip base_df and ensure student_id exists
        try:
            merged_df = pd.merge(
                merged_df, df,
                on='student_id',
                how='outer',
                suffixes=('', '_drop')
            )
        except Exception as e:
            print(f"Error merging dataframe: {e}")
            print(f"Columns in first df: {merged_df.columns.tolist()}")
            print(f"Columns in second df: {df.columns.tolist()}")

# Remove redundant columns
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_drop')]

# Create derived features with error handling
try:
    if 'last_login' in merged_df.columns and 'first_login' in merged_df.columns:
        # Ensure both columns are datetime type before subtraction
        if pd.api.types.is_datetime64_any_dtype(merged_df['last_login']) and pd.api.types.is_datetime64_any_dtype(merged_df['first_login']):
            # Calculate the difference and convert to days as integer
            merged_df['days_active'] = (merged_df['last_login'] - merged_df['first_login']).dt.total_seconds() / 86400
            merged_df['days_active'] = merged_df['days_active'].astype('float').round(1)

            if 'logins_count' in merged_df.columns:
                # Convert logins_count to numeric if needed
                if not pd.api.types.is_numeric_dtype(merged_df['logins_count']):
                    merged_df['logins_count'] = pd.to_numeric(merged_df['logins_count'], errors='coerce')

                # Now calculate engagement score
                merged_df['engagement_score'] = merged_df['logins_count'] / merged_df['days_active'].clip(lower=1)

    if 'score' in merged_df.columns:
        merged_df['performance_category'] = pd.cut(
            merged_df['score'],
            bins=[0, 60, 70, 80, 90, 100],
            labels=['F', 'D', 'C', 'B', 'A']
        )
except Exception as e:
    print(f"Error creating derived features: {e}")



## Validating

In [52]:
# Consistency checks
def validate_data(df):
    validation_issues = []

    # Make a copy to avoid modifying the original
    df = df.copy()

    # Check value ranges for score column
    if 'score' in df.columns:
        # Convert score to numeric if it's not already
        df['score'] = pd.to_numeric(df['score'], errors='coerce')

        valid_score_range = (df['score'] >= 0) & (df['score'] <= 100) | df['score'].isna()
        if not valid_score_range.all():
            invalid_count = (~valid_score_range).sum()
            validation_issues.append(f"Warning: {invalid_count} scores outside valid range (0-100)")

    # Check for logical consistency in login dates
    if 'last_login' in df.columns and 'first_login' in df.columns:
        # Only check rows that have both dates
        valid_rows = df['last_login'].notna() & df['first_login'].notna()

        if valid_rows.any():
            login_consistent = (df.loc[valid_rows, 'last_login'] >= df.loc[valid_rows, 'first_login']).all()
            if not login_consistent:
                inconsistent_count = (~(df.loc[valid_rows, 'last_login'] >= df.loc[valid_rows, 'first_login'])).sum()
                validation_issues.append(f"Warning: {inconsistent_count} rows have last_login earlier than first_login")

                # Identify these rows
                inconsistent_rows = df.loc[valid_rows & ~(df['last_login'] >= df['first_login'])]
                print("Inconsistent login date rows:")
                print(inconsistent_rows[['student_id', 'first_login', 'last_login']])

                # Fix the issue by swapping dates
                for idx in inconsistent_rows.index:
                    df.loc[idx, 'first_login'], df.loc[idx, 'last_login'] = df.loc[idx, 'last_login'], df.loc[idx, 'first_login']
                print("Dates swapped to fix inconsistency")

    # Check for duplicates if we have student_id and assignment_id
    if 'student_id' in df.columns and 'assignment_id' in df.columns:
        duplicate_entries = df.duplicated(subset=['student_id', 'assignment_id'], keep='first')
        if duplicate_entries.any():
            dup_count = duplicate_entries.sum()
            validation_issues.append(f"Warning: {dup_count} duplicate entries found")

            # Keep only the first occurrence of each student_id/assignment_id pair
            df = df.drop_duplicates(subset=['student_id', 'assignment_id'], keep='first')
            print(f"Removed {dup_count} duplicate entries")

    # Check for missing critical data
    if 'student_id' in df.columns:
        critical_cols = ['student_id']
        if 'score' in df.columns:
            critical_cols.append('score')
        if 'assignment_id' in df.columns:
            critical_cols.append('assignment_id')

        missing_critical = df[critical_cols].isnull().any(axis=1)
        if missing_critical.any():
            missing_count = missing_critical.sum()
            validation_issues.append(f"Warning: {missing_count} rows with missing critical data")

            # Identify these rows
            print("Rows with missing critical data:")
            print(df.loc[missing_critical, critical_cols])

    # Print validation summary
    if validation_issues:
        print("Validation Summary:")
        for issue in validation_issues:
            print(f"- {issue}")
    else:
        print("All validation checks passed!")

    return df

# Apply validation and get the final dataset
try:
    final_df = validate_data(merged_df)
    print(f"Final dataset shape: {final_df.shape}")
except Exception as e:
    print(f"Error during validation: {e}")
    print("Using merged_df without validation")
    final_df = merged_df

print(df.head(10))

Removed 16 duplicate entries
Rows with missing critical data:
   student_id  score assignment_id
16        009    NaN          A101
31        nan    NaN           NaN
Validation Summary:
- Warning: 16 duplicate entries found
- Warning: 2 rows with missing critical data
Final dataset shape: (17, 21)
                 Name assignment_id          Time Taken  score    Status  \
0    Katniss Everdeen          Q101 2022-09-20 10:15:00     45  Complete   
1       Peeta Mellark          Q101 2022-09-20 10:30:00     42  Complete   
2      Gale Hawthorne          Q101 2022-09-20 10:45:00     38  Complete   
3   Primrose Everdeen          Q101 2022-09-20 11:00:00     47  Complete   
4  Haymitch Abernathy          Q101 2022-09-20 11:15:00     35  Complete   
5       Effie Trinket          Q101 2022-09-20 11:30:00     48  Complete   
6               Cinna          Q101 2022-09-20 11:45:00     43  Complete   
7                 Rue          Q101 2022-09-20 12:00:00     41  Complete   
8              T

## Publishing

In [53]:
# Define output directory
output_dir = './cleaned_data/'

# Create the directory if it doesn't exist
try:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print(f"Output directory created/verified: {output_dir}")
except Exception as e:
    print(f"Unable to create output directory: {str(e)}")
    # Default to current directory if output_dir creation fails
    output_dir = './'

# Save clean dataset
try:
    output_file = os.path.join(output_dir, 'clean_student_performance.csv')
    final_df.to_csv(output_file, index=False)
    print(f"Clean dataset saved to: {output_file}")

    # Show a sample of the cleaned data
    print("\nSample of cleaned data:")
    print(final_df.head(3))
except Exception as e:
    print(f"Error saving clean dataset: {str(e)}")

# Create data dictionary
try:
    data_dict = pd.DataFrame({
        'column': final_df.columns,
        'data_type': final_df.dtypes.astype(str),
        'missing_values': final_df.isnull().sum(),
        'unique_values': [final_df[col].nunique() for col in final_df.columns],
        'sample_values': [str(final_df[col].dropna().sample(n=min(3, final_df[col].nunique())).tolist())[:50] + '...'
                         if not final_df[col].empty else 'No samples'
                         for col in final_df.columns]
    })

    dict_file = os.path.join(output_dir, 'data_dictionary.csv')
    data_dict.to_csv(dict_file, index=False)
    print(f"\nData dictionary saved to: {dict_file}")

    # Show the data dictionary
    print("\nData Dictionary Preview:")
    pd.set_option('display.max_colwidth', 30)
    print(data_dict.head())
    pd.reset_option('display.max_colwidth')
except Exception as e:
    print(f"Error creating data dictionary: {str(e)}")

# Documentation of transformation process
try:
    log_file = os.path.join(output_dir, 'transformation_log.txt')
    with open(log_file, 'w') as f:
        f.write("Data Wrangling Process\n")
        f.write("======================\n\n")

        f.write("1. Data Sources:\n")
        for file in data_files:
            f.write(f"   - {os.path.basename(file)}\n")

        f.write("\n2. Transformations Applied:\n")
        f.write("   - Standardized student IDs across all sources\n")
        f.write("   - Converted all timestamps to consistent datetime format\n")
        f.write("   - Handled missing values in critical fields\n")
        f.write("   - Merged data from multiple sources\n")
        f.write("   - Created derived metrics (days_active, engagement_score, performance_category)\n")
        f.write("   - Fixed inconsistencies and removed duplicates\n")

        f.write("\n3. Final Dataset Statistics:\n")
        f.write(f"   - Total records: {len(final_df)}\n")
        f.write(f"   - Total columns: {len(final_df.columns)}\n")

        f.write("\n4. Known Issues:\n")
        f.write("   - Some students may have missing assessment data\n")
        f.write("   - Login activity might be incomplete for some students\n")

    print(f"\nTransformation log saved to: {log_file}")

    # Display the log content
    print("\nTransformation Log Preview:")
    with open(log_file, 'r') as f:
        print(f.read()[:500] + "...\n(Log truncated for display)")
except Exception as e:
    print(f"Error creating transformation log: {str(e)}")

print("\nData publishing complete!")

Output directory created/verified: ./cleaned_data/
Clean dataset saved to: ./cleaned_data/clean_student_performance.csv

Sample of cleaned data:
  student_id first_name  last_name district registration_date  status  \
0        001    Katniss   Everdeen       12        2022-09-01  Active   
2        002      Peeta    Mellark       12        2022-09-01  Active   
4        003       Gale  Hawthorne       12               NaT  active   

          first_login          last_login  logins_count  avg_session_minutes  \
0 2022-09-01 08:30:25 2022-10-20 16:45:12            87                 42.3   
2 2022-09-01 09:15:43 2022-10-20 15:20:18            65                 38.9   
4                 NaT                 NaT            42                 25.7   

   ... assignment_id score     submission_time  \
0  ...          A101  87.5 2022-10-15 14:30:00   
2  ...          A101  92.3 2022-10-15 09:45:00   
4  ...          A101  76.8 2022-10-15 23:59:59   

                               feedback 